# Loopy IPython Integration Demo

In [ ]:
%load_ext loopy.ipython_ext

## Without transform code

In [ ]:
%%fortran_kernel

subroutine fill(out, a, n)
  implicit none

  real*8 a, out(n)
  integer n, i

  do i = 1, n
    out(i) = a
  end do
end

In [ ]:
print(prog)

## With transform code

In [ ]:
split_amount = 128

In [ ]:
%%transformed_fortran_kernel

subroutine tr_fill(out, a, n)
  implicit none

  real*8 a, out(n)
  integer n, i

  do i = 1, n
    out(i) = a
  end do
end

!$loopy begin
!
! tr_fill = lp.parse_fortran(SOURCE)
! tr_fill = lp.split_iname(tr_fill, "i", split_amount,
!     outer_tag="g.0", inner_tag="l.0")
! RESULT = tr_fill
!
!$loopy end

In [ ]:
print(prog)